# Notebook Client

This notebook provides a web-based client using matplotlib.

In [ ]:
%pylab inline
from mmf_setup.set_path import hgroot
from importlib import reload
from super_hydro.client import notebook; reload(notebook);

In [ ]:
app = notebook.get_app()
app.run()